In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
# Processing Images

clear_sky_images = os.listdir("resources/cloud-images/cloud-classifier-1/clear_sky")
cloud_images = os.listdir("resources/cloud-images/cloud-classifier-1/cloud")

labeled_data = {}

for i in clear_sky_images:
    labeled_data[i] =  [{'label': 'clear_sky'}]

for i in cloud_images:
    labeled_data[i] =  [{'label': 'cloud'}]

train_images = []
train_labels = []
class_object = ['clear_sky', 'cloud']

start_datetime = datetime.now()

for (i, image_file) in enumerate(labeled_data):
    path = 'resources/cloud-images/cloud-classifier-1/'+ labeled_data[image_file][0]['label'] + '/' + image_file
    print(path)
    image = cv2.imread(path)
    plt.imshow(image)
    plt.show()
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    label = class_object.index(labeled_data[image_file][0]['label'])
    tmp_label = labeled_data[image_file][0]['label']
    image = cv2.resize(image, (32, 32))
    pixels = image.flatten()
    train_images.append(pixels)
    train_labels.append(label)
    print('Loaded...', '\U0001F483', 'Image', str(i+1), 'is a', tmp_label)
    plt.imshow(image)
    plt.show()

end_datetime = datetime.now()

In [9]:
# Splitting the data

print('Image Processing Duration: ' + str(end_datetime-start_datetime))

train_images = np.array(train_images)
train_labels = np.array(train_labels)


test_size = 0.3

train_samples, test_samples, train_samples_labels, test_labels = train_test_split(
    train_images, train_labels, test_size=test_size, random_state=None)

Image Processing Duration: 0:00:56.389194


In [10]:
# Logistic Regression from scratch (1)

class LogisticRegressionScratch:
    def __init__(self, learning_rate=0.01, iterations=100):
        self.lr = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = 0
        self.cost_history = []

    def sigmoid(self, z):
        """Sigmoid activation function"""
        return 1 / (1 + np.exp(-z))

    def cost(self, h, y):
        """Cross-entropy loss"""
        m = len(y)
        return - (1/m) * np.sum(y*np.log(h) + (1-y)*np.log(1-h))

    def fit(self, X, y):
        """Train model using gradient descent"""
        m, n = X.shape
        self.weights = np.zeros(n)

        for _ in range(self.iterations):
            z = np.dot(X, self.weights) + self.bias
            h = self.sigmoid(z)

            dw = (1/m) * np.dot(X.T, (h - y))
            db = (1/m) * np.sum(h - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

            self.cost_history.append(self.cost(h, y))

    def predict(self, X):
        """Make predictions"""
        return (self.sigmoid(np.dot(X, self.weights) + self.bias) >= 0.5).astype(int)

In [ ]:
# Logistic Regression from scratch (3)

# Training
model = LogisticRegressionScratch(learning_rate=0.1, iterations=1000)
model.fit(train_samples, train_samples_labels)

# Evaluating accuracy
predictions = model.predict(test_samples)
accuracy = np.mean(predictions == test_labels)
print(f"Model Accuracy: {accuracy:.2f}")


/var/folders/n5/g_2w_9497mb0cvvg5_p2v3km0000gn/T/ipykernel_13482/993521626.py:18: RuntimeWarning: divide by zero encountered in log
  return - (1/m) * np.sum(y*np.log(h) + (1-y)*np.log(1-h))
/var/folders/n5/g_2w_9497mb0cvvg5_p2v3km0000gn/T/ipykernel_13482/993521626.py:18: RuntimeWarning: invalid value encountered in multiply
  return - (1/m) * np.sum(y*np.log(h) + (1-y)*np.log(1-h))
/var/folders/n5/g_2w_9497mb0cvvg5_p2v3km0000gn/T/ipykernel_13482/993521626.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Model Accuracy: 0.69


In [ ]:
# Logistic Regression with sklearn (1)
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
X_train = scaler.fit_transform(train_images)
X_test = scaler.transform(test_samples)

y_train, y_test = train_labels, test_labels

clf = LogisticRegression(C=0.01, penalty="l1", solver="saga", tol=0.1)
clf.fit(X_train, y_train)
sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(X_test, y_test)

print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)


Sparsity with L1 penalty: 98.93%
Test score with L1 penalty: 0.6667


/Users/lucasnseyep/code/LucasNseyep/belle-weder/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/lucasnseyep/code/LucasNseyep/belle-weder/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/lucasnseyep/code/LucasNseyep/belle-weder/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
